# Notebook 05: Advanced Workflows & Model Fitting

This is where we tackle more sophisticated image-processing scenarios.

**In this notebook, you will:**
- Generate images using parametric data sources (e.g., 2D Gaussian generator)
- Play through image stacks with a stack player
- Explore model fitting with `PolynomialFittingModel`
- Implement `ModelFittingContextOperation`
- Combine feature extraction and fitting in a multi-node pipeline

You’ll see how to build end-to-end systems that generate data, analyze features, and fit models—all within Semantiva’s flexible architecture.

---

1. **Parametric Data Sources**
    
    - `ParametricImageStackGenerator`: 2D Gaussian generator
    - Viewing parametric stacks (image stack player)
2. **Fitting Models**
    
    - PolynomialFittingModel
    - ModelFittingContextOperation
3. **End-to-End Feature Extraction & Fitting**
    
    - Designing a multi-node pipeline (`ProbeNode` + `ContextOperationNode`)
    - Combining parametric data generation, feature extraction, and fitting

> **Goal**: Illustrate sophisticated data generation and fitting scenarios, equipping users with tools for advanced image analysis projects.